## 1. Load up datasets for 3 months
I will be working with first quarter 2024 data. Would have preferred to load more but not sure my PC can handle the volume.

In [16]:
#load parquet file into dataframe
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import dask.dataframe as dd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error
import gc


In [10]:
def process_parquet(parquet_file_path):
    columns_needed = ['request_datetime', 'on_scene_datetime', 'PULocationID', 'DOLocationID', 'trip_time']
    
    # Read the specific columns
    df = pd.read_parquet(parquet_file_path, columns=columns_needed)
    
    # Convert datetime columns
    df['request_datetime'] = pd.to_datetime(df['request_datetime'])
    df['on_scene_datetime'] = pd.to_datetime(df['on_scene_datetime'])
    
    # Calculate the wait_time in seconds
    df['wait_time'] = (df['on_scene_datetime'] - df['request_datetime']).dt.total_seconds()
    
    return df

In [11]:
#Jan 2024
trip_jan = process_parquet('data/fhvhv_tripdata_2024-01.parquet')
trip_jan.head()

,request_datetime,on_scene_datetime,PULocationID,DOLocationID,trip_time,wait_time
0,2024-01-01 00:21:47,2024-01-01 00:25:06,161,158,2251,199.0
1,2024-01-01 00:10:56,2024-01-01 00:11:08,137,79,432,12.0
2,2024-01-01 00:20:04,2024-01-01 00:21:51,79,186,731,107.0
3,2024-01-01 00:35:46,2024-01-01 00:39:59,234,148,930,253.0
4,2024-01-01 00:48:19,2024-01-01 00:56:23,148,97,761,484.0


In [12]:
#Feb 2024
trip_feb = process_parquet('data/fhvhv_tripdata_2024-02.parquet') 
trip_feb.head()

,request_datetime,on_scene_datetime,PULocationID,DOLocationID,trip_time,wait_time
0,2024-02-01 00:13:55,2024-02-01 00:19:59,149,210,420,364.0
1,2024-02-01 00:00:14,2024-02-01 00:03:28,127,136,300,194.0
2,2024-02-01 00:13:57,2024-02-01 00:15:44,127,69,848,107.0
3,2024-02-01 00:40:21,NaT,169,169,541,NaN
4,2024-02-01 00:14:14,2024-02-01 00:15:55,142,152,658,101.0


In [13]:
#Mar 2024
trip_mar = process_parquet('data/fhvhv_tripdata_2024-03.parquet') 
trip_mar.head()

,request_datetime,on_scene_datetime,PULocationID,DOLocationID,trip_time,wait_time
0,2024-03-01 00:42:28,2024-03-01 00:45:18,148,144,170,170.0
1,2024-03-01 00:54:31,2024-03-01 00:55:55,79,114,291,84.0
2,2024-03-01 00:18:51,2024-03-01 00:20:27,232,148,425,96.0
3,2024-03-01 00:24:09,2024-03-01 00:30:33,148,233,766,384.0
4,2024-03-01 00:05:27,2024-03-01 00:07:42,48,237,1688,135.0


In [14]:
#merge the three dataframes, ensure the merge is optimised to run faster
trips = pd.concat([trip_jan, trip_feb, trip_mar], ignore_index=True)

trips.head()

,request_datetime,on_scene_datetime,PULocationID,DOLocationID,trip_time,wait_time
0,2024-01-01 00:21:47,2024-01-01 00:25:06,161,158,2251,199.0
1,2024-01-01 00:10:56,2024-01-01 00:11:08,137,79,432,12.0
2,2024-01-01 00:20:04,2024-01-01 00:21:51,79,186,731,107.0
3,2024-01-01 00:35:46,2024-01-01 00:39:59,234,148,930,253.0
4,2024-01-01 00:48:19,2024-01-01 00:56:23,148,97,761,484.0


In [17]:
#for memory sakes, I'm manually deleting the unneeded dataframes
del trip_jan
del trip_feb
del trip_mar
gc.collect()

250

## 2. Data Preprocessing
### 2.1. Calculate total trip time and total fare to dataset

In [ ]:
#create new dataframe with some columns in the trip_weather_df for modelling
new_trip_df = trips[['request_datetime', 'on_scene_datetime', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_time', 
                               'base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips']]

new_trip_df.isnull().sum().compute()

request_datetime              0
on_scene_datetime       5218737
pickup_datetime               0
dropoff_datetime              0
PULocationID                  0
DOLocationID                  0
trip_time                     0
base_passenger_fare           0
tolls                         0
bcf                           0
sales_tax                     0
congestion_surcharge          0
airport_fee                   0
tips                          0
dtype: int64

In [ ]:
#check dask dataframe shape
shape = new_trip_df.shape
shape[0].compute(),shape[1]

(19663930, 14)

In [ ]:
#wait time in seconds
wait_time = (new_trip_df['request_datetime'] - new_trip_df['on_scene_datetime']).dt.total_seconds()

#total trip time in seconds
new_trip_df['total_trip_time'] = wait_time + new_trip_df['trip_time']

#calculate total fare
new_trip_df['total_fare'] = new_trip_df['base_passenger_fare'] + new_trip_df['tolls'] + new_trip_df['bcf'] + new_trip_df['sales_tax'] + new_trip_df['congestion_surcharge'] + new_trip_df['airport_fee'] + new_trip_df['tips']

new_trip_df.head()

### 2.2 Drop unnecessary columns

In [ ]:
#remove unnecessary columns
new_trip_df = new_trip_df.drop(['base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips', 'on_scene_datetime', 'pickup_datetime', 'dropoff_datetime', 'trip_time'], axis=1)

new_trip_df.head()

### 2.3. Investigate all columns (nulls and duplicates)

In [ ]:
new_trip_df.head()

In [ ]:
#check for null in dataframe
new_trip_df.isnull().sum().compute()


### 2.4. Grouping and Loading additional data
#### 2.4.1. Peak, off-peak and night time

In [ ]:
""" 
    Group request datetime into Peak, Off-Peak and Night
"""

#Function to group the request_datetime as Peak:2, OffPeak:1 or Night:0
def group_request_time(fhvhv_row):
    day_of_week = fhvhv_row['request_datetime'].weekday()  # Monday=0, Sunday=6
    hour = fhvhv_row['request_datetime'].hour
    night, off_peak, peak = 0, 1, 2

    if hour >= 23 or hour < 6:
        return night
    elif day_of_week < 5:  # Monday to Friday
        if (6 <= hour < 10) or (15 <= hour < 19):
            return peak
        elif (10 <= hour < 15) or (19 <= hour < 23):
            return off_peak
    else:  # Saturday and Sunday
        if hour >= 6 and hour < 23:
            return off_peak
        else:
            return night

#apply function to dataframe
new_trip_df['request_time_group'] = new_trip_df.apply(group_request_time, axis=1, meta=('request_time_group', object))

new_trip_df.head()

#### 2.4.2. Holidays in NYC

In [ ]:
#Holidays
from datetime import date
import holidays

ny_holidays = holidays.country_holidays('US', subdiv='NY', years=2024)

#load ny_holidays into dataframe, date and holiday name
ny_holidays_df = pd.DataFrame.from_dict(ny_holidays, orient='index')
ny_holidays_df

In [ ]:
#reset index and rename columns
ny_holidays_df.reset_index(inplace=True)
ny_holidays_df.columns = ['date', 'holiday_name']
ny_holidays_df

In [ ]:
#convert holidays to datetime and create is_holiday flag in the trips dataframe
ny_holidays_df.index = pd.to_datetime(ny_holidays_df.index)
new_trip_df['is_holiday'] = new_trip_df['request_datetime'].dt.date.isin(ny_holidays_df['date']).astype(int)
new_trip_df.head()

#### 2.4.3. NYC weather

In [ ]:
#Get Weather data
from datetime import datetime 
from meteostat import Point, Monthly, Daily, Hourly, Normals, units  
# Set time period 
start = datetime(2024, 1, 1) 
end = datetime(2024, 3, 31)  
# Create Point for NY
location = Point(40.712775, -74.005973)  
# Get daily data 
weather_data = Hourly(location, start, end) 
weather_data = weather_data.convert(units.imperial) 
weather_data = weather_data.fetch()

weather_data.head()

In [ ]:
#reset index and convert time to datetime
weather_data.reset_index(inplace=True)
weather_data.rename(columns={'index': 'time'}, inplace=True)
weather_data['time'] = pd.to_datetime(weather_data['time'])

weather_data.head()

In [ ]:
#Select the required columns from weather data and merge with trips dataframe
weather_data = weather_data[['time', 'temp', 'coco']]

#cast request_datetime to datetime64[ns] to stop warnings
new_trip_df['request_datetime'] = new_trip_df['request_datetime'].astype('datetime64[ns]')
trip_weather_df = new_trip_df.merge(weather_data, left_on='request_datetime', right_on='time').compute()

#drop time column
trip_weather_df.drop('time', axis=1, inplace=True)

trip_weather_df.head()

## 3. EDA
### i. Check for nulls, duplicates

In [ ]:
#print shape of dataframe
trip_weather_df.shape

#check for nulls in trip_weather_df
trip_weather_df.isnull().sum()

In [ ]:
#check for duplicated values
trip_weather_df.duplicated().sum()

In [ ]:
#drop nulls
trip_weather_df.dropna(inplace=True)

#check shape
trip_weather_df.shape

#check for nulls
trip_weather_df.isnull().sum()

In [ ]:
#create new dataframe with some columns in the trip_weather_df for modelling
new_trip_df = trip_weather_df[['request_datetime', 'on_scene_datetime', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_time', 
                               'base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips','request_time_group', 'is_holiday', 'temp', 'coco']]

#wait time in seconds
new_trip_df['wait_time'] = (new_trip_df['request_datetime'] - new_trip_df['on_scene_datetime']).dt.total_seconds()

#total trip time in seconds
new_trip_df['total_trip_time'] = new_trip_df['wait_time'] + new_trip_df['trip_time']

#calculate total fare
new_trip_df['total_fare'] = new_trip_df['base_passenger_fare'] + new_trip_df['tolls'] + new_trip_df['bcf'] + new_trip_df['sales_tax'] + new_trip_df['congestion_surcharge'] + new_trip_df['airport_fee'] + new_trip_df['tips']

#remove unnecessary columns
new_trip_df.drop(['base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips', 'request_datetime', 'on_scene_datetime', 'pickup_datetime', 'dropoff_datetime'], axis=1, inplace=True)

new_trip_df.head()

In [ ]:
#import seaborn for pairplot
import seaborn as sns
g = sns.pairplot(new_trip_df)

In [ ]:
sns.set(rc={'figure.figsize':(15,10)})

n_variables = ['PULocationID', 'DOLocationID', 'trip_time', 'request_time_group', 'is_holiday', 'temp', 'coco', 'wait_time', 'total_trip_time', 'total_fare']

pc = new_trip_df[n_variables].corr(method ='pearson')

cols = n_variables

ax = sns.heatmap(pc, annot=True,
                 yticklabels=cols,
                 xticklabels=cols,
                 annot_kws={'size':10},
                 cmap="Blues")
